In [1]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import csv
import optuna

In [2]:
# 셀에서 출력 정확도 설정.
%precision %.3f

'%.3f'

In [3]:
file_path = r'.\crawling_data\preprocessed_essay.csv'

# 자소서 리스트화
documents = []
with open(file_path, 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        documents.append(row[0])

In [4]:
# 전문가 평점 1~5점 각 15개씩 랜덤으로 가져와서 점수 라벨링
# 회귀 예측용 label
labels = []
for i in range(317):
    if i < 22:
        labels.append(5)
    elif i < 77:
        labels.append(4)
    elif i < 175:
        labels.append(3)
    elif i < 302:
        labels.append(2)        
    else:
        labels.append(1)

In [5]:
# 불용어 리스트 가져오기
stop_words = set(stopwords.words('english'))

def preprocess(text):
    tokens = word_tokenize(text)
    filtered_tokens = [word for word in tokens if word.isalpha() and word not in stop_words]
    return filtered_tokens

documents_corpus = []
for document in documents:
    documents_corpus.append(preprocess(document))

In [6]:
X_train, X_test, y_train, y_test = train_test_split(documents_corpus, labels, test_size=0.3, random_state=123)

In [7]:
# Tagged Corpus 생성.
# 모델 학습에 필요한 형태로 변환.
my_tagged_corpus = [TaggedDocument(words=my_words, tags=[i]) for i, my_words in enumerate(X_train)]

In [8]:
# Optuna 목적 함수
def objective(trial):
    # Doc2Vec 하이퍼파라미터 튜닝
    vector_size = trial.suggest_int('vector_size', 50, 300, step=10)
    window = trial.suggest_int('window', 3, 15)
    min_count = trial.suggest_int('min_count', 1, 20)
    epochs = trial.suggest_int('epochs', 20, 50, step=5)
    dm = trial.suggest_int('dm', 0, 1)

    # Doc2Vec 모델 학습
    model = Doc2Vec(my_tagged_corpus, vector_size=vector_size, window=window, min_count=min_count, epochs=epochs, dm=dm, seed=123)
    X_train_vec = [model.infer_vector(doc) for doc in X_train]
    X_test_vec = [model.infer_vector(doc) for doc in X_test]

    # Randomforest 하이퍼파라미터 튜닝
    n_estimators = trial.suggest_int('n_estimators', 50, 500)
    max_depth = trial.suggest_int('max_depth', 5, 30)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 5)
    max_features = trial.suggest_float('max_features', 0.1, 1.0)

    # Randomforest 모델 생성
    model = RandomForestRegressor(n_estimators=n_estimators,
                                    max_depth=max_depth,
                                    min_samples_split=min_samples_split,
                                    min_samples_leaf=min_samples_leaf,
                                    max_features=max_features,
                                    random_state=123)
    
    model.fit(X_train_vec, y_train)
    score = root_mean_squared_error(y_test, model.predict(X_test_vec))
    
    return score

# Optuna 최적화 수행
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100, n_jobs=-1)

# 최적 하이퍼파라미터 출력
print('Best hyperparameters: ', study.best_params)

[I 2024-06-20 20:14:20,273] A new study created in memory with name: no-name-53b66122-3f3b-44a6-9fe0-687038c2e426
[I 2024-06-20 20:14:43,649] Trial 2 finished with value: 1.0010182261448282 and parameters: {'vector_size': 120, 'window': 8, 'min_count': 13, 'epochs': 30, 'dm': 0, 'n_estimators': 154, 'max_depth': 14, 'min_samples_split': 3, 'min_samples_leaf': 2, 'max_features': 0.25152742471937595}. Best is trial 2 with value: 1.0010182261448282.
[I 2024-06-20 20:14:52,410] Trial 7 finished with value: 0.9828163905400825 and parameters: {'vector_size': 210, 'window': 15, 'min_count': 11, 'epochs': 40, 'dm': 0, 'n_estimators': 447, 'max_depth': 20, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 0.1627989097260507}. Best is trial 7 with value: 0.9828163905400825.
[I 2024-06-20 20:14:53,842] Trial 3 finished with value: 0.9741112846287654 and parameters: {'vector_size': 240, 'window': 10, 'min_count': 12, 'epochs': 45, 'dm': 0, 'n_estimators': 221, 'max_depth': 5, 'min_sam

Best hyperparameters:  {'vector_size': 170, 'window': 5, 'min_count': 12, 'epochs': 20, 'dm': 1, 'n_estimators': 479, 'max_depth': 20, 'min_samples_split': 5, 'min_samples_leaf': 3, 'max_features': 0.5199648225366829}


In [9]:
best_params = study.best_params

# Doc2Vec 모델 생성
model = Doc2Vec(my_tagged_corpus, vector_size=best_params['vector_size'], 
                window=best_params['window'], min_count=best_params['min_count'],
                epochs=best_params['epochs'], dm=best_params['dm'], seed=123)

# RandomForest 모델 생성
rg = RandomForestRegressor(n_estimators=best_params['n_estimators'],
                                    max_depth=best_params['max_depth'],
                                    min_samples_split=best_params['min_samples_split'],
                                    min_samples_leaf=best_params['min_samples_leaf'],
                                    max_features=best_params['max_features'],
                                    random_state=123)

# Doc2Vec 벡터 생성
X_train_vec = [model.infer_vector(doc) for doc in X_train]
X_test_vec = [model.infer_vector(doc) for doc in X_test]

# 분류 모델 학습 및 예측
rg.fit(X_train_vec, y_train)
y_pred = rg.predict(X_test_vec)

print('# RandomForestRegressor')
print(r2_score(y_test, y_pred))
print(root_mean_squared_error(y_test, y_pred))

# RandomForestRegressor
0.23137637900591002
0.8850436131002039


In [10]:
# Optuna 목적 함수
def objective(trial):
    # Doc2Vec 하이퍼파라미터 튜닝
    vector_size = trial.suggest_int('vector_size', 50, 300, step=10)
    window = trial.suggest_int('window', 3, 15)
    min_count = trial.suggest_int('min_count', 1, 20)
    epochs = trial.suggest_int('epochs', 20, 50, step=5)
    dm = trial.suggest_int('dm', 0, 1)

    # Doc2Vec 모델 학습
    model = Doc2Vec(my_tagged_corpus, vector_size=vector_size, window=window, min_count=min_count, epochs=epochs, dm=dm, seed=123)
    X_train_vec = [model.infer_vector(doc) for doc in X_train]
    X_test_vec = [model.infer_vector(doc) for doc in X_test]

    # SVM 하이퍼파라미터 튜닝
    C = trial.suggest_float('C', 0.01, 100.0, log=True)
    kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly', 'sigmoid'])
    degree = trial.suggest_int('degree', 2, 5) if kernel == 'poly' else 3
    gamma = trial.suggest_float('gamma', 1e-4, 1.0, log=True) if kernel in ['rbf', 'poly'] else 'scale'

    # SVM 모델 생성
    model = SVR(
        C=C,
        kernel=kernel,
        degree=degree,
        gamma=gamma
    )

    model.fit(X_train_vec, y_train)
    score = root_mean_squared_error(y_test, model.predict(X_test_vec))
    
    return score

# Optuna 최적화 수행
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100, n_jobs=-1)

# 최적 하이퍼파라미터 출력
print('Best hyperparameters: ', study.best_params)

[I 2024-06-20 20:22:47,240] A new study created in memory with name: no-name-e8f44c9e-f326-4ffe-8f52-1874d016b577
[I 2024-06-20 20:23:10,006] Trial 7 finished with value: 1.0083212597447988 and parameters: {'vector_size': 300, 'window': 8, 'min_count': 5, 'epochs': 25, 'dm': 0, 'C': 0.20845724822457934, 'kernel': 'rbf', 'gamma': 0.0004044313472284562}. Best is trial 7 with value: 1.0083212597447988.
[I 2024-06-20 20:23:14,981] Trial 4 finished with value: 0.9336616338472271 and parameters: {'vector_size': 240, 'window': 7, 'min_count': 14, 'epochs': 35, 'dm': 0, 'C': 0.4122192030340788, 'kernel': 'sigmoid'}. Best is trial 4 with value: 0.9336616338472271.
[I 2024-06-20 20:23:15,123] Trial 1 finished with value: 1.0072909729195754 and parameters: {'vector_size': 210, 'window': 4, 'min_count': 13, 'epochs': 35, 'dm': 0, 'C': 0.017968570317083363, 'kernel': 'sigmoid'}. Best is trial 4 with value: 0.9336616338472271.
[I 2024-06-20 20:23:16,098] Trial 6 finished with value: 1.00129257030399

Best hyperparameters:  {'vector_size': 220, 'window': 5, 'min_count': 16, 'epochs': 45, 'dm': 1, 'C': 35.765703800299256, 'kernel': 'rbf', 'gamma': 0.00902505403709979}


In [11]:
best_params = study.best_params

# Doc2Vec 모델 생성
model = Doc2Vec(my_tagged_corpus, vector_size=best_params['vector_size'], 
                window=best_params['window'], min_count=best_params['min_count'],
                epochs=best_params['epochs'], dm=best_params['dm'], seed=123)

# SVM 분류 모델 생성
rg = SVR(C=best_params['C'], kernel=best_params['kernel'])#degree=best_params['degree'], gamma=best_params['gamma'], 

# Doc2Vec 벡터 생성
X_train_vec = [model.infer_vector(doc) for doc in X_train]
X_test_vec = [model.infer_vector(doc) for doc in X_test]

# 회귀 모델 학습 및 예측
rg.fit(X_train_vec, y_train)
y_pred = rg.predict(X_test_vec)

print('# SVM Regressor')
print(r2_score(y_test, y_pred))
print(root_mean_squared_error(y_test, y_pred))

# SVM Regressor
0.26163058331182165
0.8674504144449785


In [12]:
# Optuna 목적 함수
def objective(trial):
    # Doc2Vec 하이퍼파라미터 튜닝
    vector_size = trial.suggest_int('vector_size', 50, 300, step=10)
    window = trial.suggest_int('window', 3, 15)
    min_count = trial.suggest_int('min_count', 1, 20)
    epochs = trial.suggest_int('epochs', 20, 50, step=5)
    dm = trial.suggest_int('dm', 0, 1)

    # Doc2Vec 모델 학습
    model = Doc2Vec(my_tagged_corpus, vector_size=vector_size, window=window, min_count=min_count, epochs=epochs, dm=dm, seed=123)
    X_train_vec = [model.infer_vector(doc) for doc in X_train]
    X_test_vec = [model.infer_vector(doc) for doc in X_test]

    # XGBOOST 하이퍼파라미터 튜닝
    max_depth = trial.suggest_int('max_depth', 2, 30)
    n_estimators = trial.suggest_int('n_estimators', 100, 500)
    learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3)
    gamma = trial.suggest_float('gamma', 0, 1)
    min_child_weight = trial.suggest_int('min_child_weight', 1, 20)
    
    # XGboost 모델 생성
    model = XGBRegressor(max_depth=max_depth,
                        n_estimators=n_estimators,
                        learning_rate=learning_rate,
                        gamma=gamma,
                        min_child_weight=min_child_weight,
                        random_state=123)
    
    model.fit(X_train_vec, y_train)
    score = root_mean_squared_error(y_test, model.predict(X_test_vec))
    
    return score

# Optuna 최적화 수행
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100, n_jobs=-1)

# 최적 하이퍼파라미터 출력
print('Best hyperparameters: ', study.best_params)

[I 2024-06-20 20:30:05,392] A new study created in memory with name: no-name-598f4002-f7c1-4713-9a44-6ec46e7a32c9
[I 2024-06-20 20:30:32,624] Trial 6 finished with value: 0.9594877704496285 and parameters: {'vector_size': 300, 'window': 8, 'min_count': 14, 'epochs': 35, 'dm': 0, 'max_depth': 14, 'n_estimators': 220, 'learning_rate': 0.19324049315291997, 'gamma': 0.45498117307220265, 'min_child_weight': 10}. Best is trial 6 with value: 0.9594877704496285.
[I 2024-06-20 20:30:39,528] Trial 5 finished with value: 1.018380583370236 and parameters: {'vector_size': 170, 'window': 13, 'min_count': 18, 'epochs': 50, 'dm': 0, 'max_depth': 26, 'n_estimators': 277, 'learning_rate': 0.10002778044372855, 'gamma': 0.7445091569776981, 'min_child_weight': 7}. Best is trial 6 with value: 0.9594877704496285.
[I 2024-06-20 20:30:41,632] Trial 4 finished with value: 1.0151737668686296 and parameters: {'vector_size': 100, 'window': 14, 'min_count': 3, 'epochs': 50, 'dm': 0, 'max_depth': 14, 'n_estimators':

Best hyperparameters:  {'vector_size': 130, 'window': 3, 'min_count': 4, 'epochs': 20, 'dm': 1, 'max_depth': 17, 'n_estimators': 317, 'learning_rate': 0.04537551620843587, 'gamma': 0.831903407107819, 'min_child_weight': 13}


In [13]:
best_params = study.best_params

# Doc2Vec 모델 생성
model = Doc2Vec(my_tagged_corpus, vector_size=best_params['vector_size'], 
                window=best_params['window'], min_count=best_params['min_count'],
                epochs=best_params['epochs'], dm=best_params['dm'], seed=123)

# XGBoost 모델 생성
rg = XGBRegressor(
    max_depth=best_params['max_depth'],
    n_estimators=best_params['n_estimators'],
    learning_rate=best_params['learning_rate'],
    gamma=best_params['gamma'],
    min_child_weight=best_params['min_child_weight'],
    random_state=123
)

# Doc2Vec 벡터 생성
X_train_vec = [model.infer_vector(doc) for doc in X_train]
X_test_vec = [model.infer_vector(doc) for doc in X_test]

# 회귀 모델 학습 및 예측
rg.fit(X_train_vec, y_train)
y_pred = rg.predict(X_test_vec)

print('# XGBoost Regressor')
print(r2_score(y_test, y_pred))
print(root_mean_squared_error(y_test, y_pred))

# XGBoost Regressor
0.22242581844329834
0.8901818640079227


In [14]:
# Optuna 목적 함수
def objective(trial):
    # Doc2Vec 하이퍼파라미터 튜닝
    vector_size = trial.suggest_int('vector_size', 50, 300, step=10)
    window = trial.suggest_int('window', 3, 15)
    min_count = trial.suggest_int('min_count', 1, 20)
    epochs = trial.suggest_int('epochs', 20, 50, step=5)
    dm = trial.suggest_int('dm', 0, 1)

    # Doc2Vec 모델 학습
    model = Doc2Vec(my_tagged_corpus, vector_size=vector_size, window=window, min_count=min_count, epochs=epochs, dm=dm, seed=123)
    X_train_vec = [model.infer_vector(doc) for doc in X_train]
    X_test_vec = [model.infer_vector(doc) for doc in X_test]

    # LightGBM 하이퍼파라미터 튜닝
    max_depth = trial.suggest_int('max_depth', 2, 30)
    num_leaves = trial.suggest_int('num_leaves', 10, 200)
    learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3)
    n_estimators = trial.suggest_int('n_estimators', 50, 500)
    min_child_samples = trial.suggest_int('min_child_samples', 3, 50)

    # LightGBM 모델 생성
    model = LGBMRegressor(
        max_depth=max_depth,
        num_leaves=num_leaves,
        learning_rate=learning_rate,
        n_estimators=n_estimators,
        min_child_samples=min_child_samples,
        random_state=123
    )
    
    model.fit(X_train_vec, y_train)
    score = root_mean_squared_error(y_test, model.predict(X_test_vec))
    
    return score

# Optuna 최적화 수행
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100, n_jobs=-1)

# 최적 하이퍼파라미터 출력
print('Best hyperparameters: ', study.best_params)

[I 2024-06-20 20:39:48,372] A new study created in memory with name: no-name-c0539785-f279-4642-b095-4802b943b38a
[I 2024-06-20 20:40:20,683] Trial 7 finished with value: 1.0509842448403175 and parameters: {'vector_size': 120, 'window': 3, 'min_count': 20, 'epochs': 40, 'dm': 0, 'max_depth': 3, 'num_leaves': 189, 'learning_rate': 0.18065201707315784, 'n_estimators': 223, 'min_child_samples': 14}. Best is trial 7 with value: 1.0509842448403175.
[I 2024-06-20 20:40:21,786] Trial 4 finished with value: 1.058305394018947 and parameters: {'vector_size': 180, 'window': 13, 'min_count': 12, 'epochs': 40, 'dm': 0, 'max_depth': 11, 'num_leaves': 59, 'learning_rate': 0.04040099538184303, 'n_estimators': 225, 'min_child_samples': 25}. Best is trial 7 with value: 1.0509842448403175.
[I 2024-06-20 20:40:24,722] Trial 6 finished with value: 1.0816792950859355 and parameters: {'vector_size': 220, 'window': 15, 'min_count': 18, 'epochs': 45, 'dm': 0, 'max_depth': 24, 'num_leaves': 89, 'learning_rate':

Best hyperparameters:  {'vector_size': 220, 'window': 10, 'min_count': 3, 'epochs': 25, 'dm': 1, 'max_depth': 18, 'num_leaves': 92, 'learning_rate': 0.02058165766443268, 'n_estimators': 221, 'min_child_samples': 29}


In [15]:
best_params = study.best_params

# Doc2Vec 모델 생성
model = Doc2Vec(my_tagged_corpus, vector_size=best_params['vector_size'], 
                window=best_params['window'], min_count=best_params['min_count'],
                epochs=best_params['epochs'], dm=best_params['dm'], seed=123)

# LightGBM 모델 생성
rg = LGBMRegressor(max_depth=best_params['max_depth'], num_leaves=best_params['num_leaves'], 
                    learning_rate=best_params['learning_rate'], n_estimators=best_params['n_estimators'], 
                    min_child_samples=best_params['min_child_samples'], random_state=123)


# Doc2Vec 벡터 생성
X_train_vec = [model.infer_vector(doc) for doc in X_train]
X_test_vec = [model.infer_vector(doc) for doc in X_test]

# 회귀 모델 학습 및 예측
rg.fit(X_train_vec, y_train)
y_pred = rg.predict(X_test_vec)

print('# LightGBM Regressor')
print(r2_score(y_test, y_pred))
print(root_mean_squared_error(y_test, y_pred))

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000659 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 16567
[LightGBM] [Info] Number of data points in the train set: 221, number of used features: 220
[LightGBM] [Info] Start training from score 2.828054
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,

In [16]:
# Optuna 목적 함수
def objective(trial):
    # Doc2Vec 하이퍼파라미터 튜닝
    vector_size = trial.suggest_int('vector_size', 50, 300, step=10)
    window = trial.suggest_int('window', 3, 15)
    min_count = trial.suggest_int('min_count', 1, 20)
    epochs = trial.suggest_int('epochs', 20, 50, step=5)
    dm = trial.suggest_int('dm', 0, 1)

    # Doc2Vec 모델 학습
    model = Doc2Vec(my_tagged_corpus, vector_size=vector_size, window=window, min_count=min_count, epochs=epochs, dm=dm, seed=123)
    X_train_vec = [model.infer_vector(doc) for doc in X_train]
    X_test_vec = [model.infer_vector(doc) for doc in X_test]

    # Linear Regression 하이퍼파라미터 튜닝
    fit_intercept = trial.suggest_categorical('fit_intercept', [True, False])
    
    # Linear Regression 모델 생성
    model = LinearRegression(fit_intercept=fit_intercept)
    
    model.fit(X_train_vec, y_train)
    score = root_mean_squared_error(y_test, model.predict(X_test_vec))
    
    return score

# Optuna 최적화 수행
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100, n_jobs=-1)

# 최적 하이퍼파라미터 출력
print('Best hyperparameters: ', study.best_params)

[I 2024-06-20 20:46:57,841] A new study created in memory with name: no-name-c5efca3e-3143-4013-9a91-4e1b7d16716f
[I 2024-06-20 20:47:22,398] Trial 7 finished with value: 1.052369314161242 and parameters: {'vector_size': 210, 'window': 7, 'min_count': 1, 'epochs': 25, 'dm': 0, 'fit_intercept': True}. Best is trial 7 with value: 1.052369314161242.
[I 2024-06-20 20:47:22,546] Trial 5 finished with value: 1.461309890046417 and parameters: {'vector_size': 240, 'window': 10, 'min_count': 1, 'epochs': 25, 'dm': 0, 'fit_intercept': False}. Best is trial 7 with value: 1.052369314161242.
[I 2024-06-20 20:47:33,328] Trial 0 finished with value: 1.1670934192464792 and parameters: {'vector_size': 130, 'window': 8, 'min_count': 18, 'epochs': 50, 'dm': 0, 'fit_intercept': True}. Best is trial 7 with value: 1.052369314161242.
[I 2024-06-20 20:47:35,069] Trial 6 finished with value: 8.09927413751898 and parameters: {'vector_size': 220, 'window': 12, 'min_count': 19, 'epochs': 20, 'dm': 1, 'fit_interce

Best hyperparameters:  {'vector_size': 90, 'window': 9, 'min_count': 17, 'epochs': 20, 'dm': 0, 'fit_intercept': True}


In [17]:
best_params = study.best_params

# Doc2Vec 모델 생성
model = Doc2Vec(my_tagged_corpus, vector_size=best_params['vector_size'], 
                window=best_params['window'], min_count=best_params['min_count'],
                epochs=best_params['epochs'], dm=best_params['dm'], seed=123)

# Linear Regression 모델 생성
rg = LinearRegression(fit_intercept=best_params['fit_intercept'])

# Doc2Vec 벡터 생성
X_train_vec = [model.infer_vector(doc) for doc in X_train]
X_test_vec = [model.infer_vector(doc) for doc in X_test]

# 회귀 모델 학습 및 예측
rg.fit(X_train_vec, y_train)
y_pred = rg.predict(X_test_vec)

print('# LinearRegression')
print(r2_score(y_test, y_pred))
print(root_mean_squared_error(y_test, y_pred))

# LinearRegression
0.13996875286102295
0.93619198062049


In [18]:
# import pickle

# with open('d2v_model_1245.pkl', 'wb') as f:
#     pickle.dump(model, f)
    
# with open('svc_model_1245.pkl', 'wb') as f:
#     pickle.dump(clf, f)